https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-groupby.html

In [1]:


import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [3]:
df = spark.sql('''select 'spark' as hello ''')  
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:

%%html
<style>
div.output_area pre {
    white-space: pre;}
</style>

In [4]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('rows.csv@accessType=DOWNLOAD',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

####################      only filtering till 2018       #######################

print(rc.show(5))       #######3 to show in  a nice format we use show

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

The sql function on a SparkSession enables applications to run SQL queries programmatically and returns the result as a DataFrame.


In [9]:
## RC= REGISTERED CRIMES
# Register the DataFrame as a SQL temporary view
rc.createOrReplaceTempView("regcrim")

sqlDF = spark.sql("SELECT * FROM regcrim")
sqlDF.show()


+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| fal

Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates. 

In [12]:
# Register the DataFrame as a global temporary view
rc.createGlobalTempView("regcrim")


sqlDF = spark.sql("SELECT * FROM regcrim")
sqlDF.show()

AnalysisException: Temporary view 'regcrim' already exists

In [13]:
sqlDF.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [15]:
spark.sql("SELECT * FROM regcrim").show(1)

+--------+-----------+-------------------+---------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|          Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+---------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     009|  12|            6

In [55]:
spark.sql("SELECT ID,`Case Number`,Description,Date FROM regcrim WHERE ID='10224738'").show()

########   names with space use  `  `   AND NOT ' '------------->  CAREFUL

+--------+-----------+--------------------+-------------------+
|      ID|Case Number|         Description|               Date|
+--------+-----------+--------------------+-------------------+
|10224738|   HY411648|DOMESTIC BATTERY ...|2015-09-05 13:30:00|
+--------+-----------+--------------------+-------------------+



In [21]:
spark.sql("SELECT * FROM regcrim WHERE ID='10224738'").show()

+--------+-----------+-------------------+---------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|          Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+---------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     009|  12|            6

In [47]:

spark.sql("SELECT * FROM regcrim ORDER BY ID").show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10000012|   HY189978|2015-03-18 21:30:00|018XX S CENTRAL P...|0486|             BATTERY|DOMESTIC BATTERY ...|           APARTM

In [54]:
spark.sql("SELECT `Location Description` FROM regcrim ").show()

+--------------------+
|Location Description|
+--------------------+
|           RESIDENCE|
|             CTA BUS|
|           RESIDENCE|
|            SIDEWALK|
|           APARTMENT|
|           RESIDENCE|
|    RESIDENCE-GARAGE|
|  GROCERY FOOD STORE|
|              STREET|
|                null|
|PARKING LOT/GARAG...|
|  SMALL RETAIL STORE|
|           APARTMENT|
|              STREET|
|              STREET|
|           APARTMENT|
|              STREET|
|               OTHER|
|           RESIDENCE|
|              STREET|
+--------------------+
only showing top 20 rows



In [56]:
spark.sql("SELECT `Primary Type`,count(`Primary Type`) FROM regcrim GROUP BY `Primary Type`").show()

+--------------------+-------------------+
|        Primary Type|count(Primary Type)|
+--------------------+-------------------+
|OFFENSE INVOLVING...|               6037|
|CRIMINAL SEXUAL A...|                 42|
|            STALKING|                460|
|PUBLIC PEACE VIOL...|               4503|
|           OBSCENITY|                173|
|NON-CRIMINAL (SUB...|                  4|
|                null|                  0|
|               ARSON|               1221|
|            GAMBLING|                541|
|   CRIMINAL TRESPASS|              17723|
|             ASSAULT|              49239|
|      NON - CRIMINAL|                 13|
|LIQUOR LAW VIOLATION|                615|
| MOTOR VEHICLE THEFT|              29396|
|               THEFT|             162737|
|             BATTERY|             131246|
|             ROBBERY|              30365|
|            HOMICIDE|                126|
|    PUBLIC INDECENCY|                 27|
| CRIM SEXUAL ASSAULT|               3953|
+----------

In [58]:
#ORDER BY Country DESC
spark.sql("SELECT `Primary Type`,count(`Primary Type`) FROM regcrim GROUP BY `Primary Type` ORDER BY count(`Primary Type`) DESC ").show(3)

+---------------+-------------------+
|   Primary Type|count(Primary Type)|
+---------------+-------------------+
|          THEFT|             162737|
|        BATTERY|             131246|
|CRIMINAL DAMAGE|              79989|
+---------------+-------------------+
only showing top 3 rows



In [ ]:
### WHAT PERCENT OF REPORTED FUNCTIONS RESULTED IN AN ARREST

TOTAL TRUE ARREST CASES / TOTAL NUMBER OF ROWS * 100

spark.sql("SELECT `Primary Type`,count(`Primary Type`) FROM regcrim GROUP BY `Primary Type`").show()

In [63]:
spark.sql("SELECT count(`Arrest`) FROM regcrim ").show()

+-------------+
|count(Arrest)|
+-------------+
|       707956|
+-------------+



In [67]:
spark.sql("SELECT count(`Arrest`) FROM regcrim WHERE Arrest= true ").show()

+-------------+
|count(Arrest)|
+-------------+
|       147453|
+-------------+



In [62]:
spark.sql("SELECT count(`Arrest`) FROM regcrim WHERE Arrest= false ").show()

+-------------+
|count(Arrest)|
+-------------+
|       560503|
+-------------+

